## 1. Question 1: **Answer**

In [1]:
import sqlite3
import csv

In [ ]:
def load_student_data(data):
    try:
        conn = sqlite3.connect("student_grades_data.db")
        cursor = conn.cursor()

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS grades (
            Student_id PRIMARY KEY,
            subject TEXT,
            score INTEGER)
            """
        )
        with open(data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO grades VALUES (?, ?, ?)", row)
        conn.commit()
        print("Data loaded succesfully")
    except sqlite3.Error as e:
        print(f"Error: {e}")
    finally:
        conn.close()

# Loading data
data = "/content/student_grades.csv"
load_student_data(data)

Data loaded succesfully


In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///student_grades_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM grades;

 * sqlite:///student_grades_data.db
Done.


Student_id,subject,score
1,Math,85
2,Science,92
3,English,78
4,History,65
5,Math,98
6,Science,80
7,English,90
8,History,60
9,Math,68
10,Science,55


## Question 2: **Answer**

In [7]:
%%sql
-- Step 1: Adding a scored_grading column
ALTER TABLE grades
ADD COLUMN score_grading TEXT;

 * sqlite:///student_grades_data.db
Done.


[]

In [8]:
%%sql
-- Step 2: Update the score_grading column based on the score
UPDATE grades
SET score_grading = CASE
                        WHEN score >= 90 THEN 'A'
                        WHEN score BETWEEN 80 AND 89 THEN 'B'
                        WHEN score BETWEEN 70 AND 79 THEN 'C'
                        WHEN score BETWEEN 60 AND 69 THEN 'D'
                        ELSE 'F'
                    END;

 * sqlite:///student_grades_data.db
10 rows affected.


[]

In [9]:
%%sql
SELECT * FROM grades LIMIT 5;

 * sqlite:///student_grades_data.db
Done.


Student_id,subject,score,score_grading
1,Math,85,B
2,Science,92,A
3,English,78,C
4,History,65,D
5,Math,98,A


## Question 3: **Answer**

In [10]:
%%sql
-- Step 1: Adding a flags column
ALTER TABLE grades
ADD COLUMN flags TEXT;

 * sqlite:///student_grades_data.db
Done.


[]

In [11]:
%%sql
-- Step 2: Update the flags column with values
UPDATE grades
SET flags =
    CASE
        WHEN score >= 75 THEN 'High performer'
        ELSE 'Not high performer'
    END;

 * sqlite:///student_grades_data.db
10 rows affected.


[]

In [12]:
%%sql
SELECT * FROM grades LIMIT 5;

 * sqlite:///student_grades_data.db
Done.


Student_id,subject,score,score_grading,flags
1,Math,85,B,High performer
2,Science,92,A,High performer
3,English,78,C,High performer
4,History,65,D,Not high performer
5,Math,98,A,High performer


In [13]:
%%sql
SELECT
    COUNT(*) AS number_of_high_perfomers
FROM grades
WHERE flags = 'High performer';

 * sqlite:///student_grades_data.db
Done.


number_of_high_perfomers
6


## Question 4: **Answer**

In [14]:
%%sql
SELECT
    subject,
    AVG(score) AS average_math_score
FROM grades
WHERE subject = 'Math' AND flags = 'High performer';

 * sqlite:///student_grades_data.db
Done.


subject,average_math_score
Math,91.5


## Question 5: **Answer**

In [15]:
%%sql
SELECT
    COUNT(*) AS excellent_count
FROM
    (SELECT
    score,
    CASE
        WHEN score >= 85 THEN 'Excellent'
        WHEN score BETWEEN 70 AND 84 THEN 'Good'
        ELSE 'Poor'
    END AS student_standing
    FROM grades) AS student_grading
WHERE student_standing = 'Excellent';

 * sqlite:///student_grades_data.db
Done.


excellent_count
4


In [16]:
# %%sql
# DROP TABLE grades;